<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подключение-к-БД-и-выгрузка-данных" data-toc-modified-id="Подключение-к-БД-и-выгрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подключение к БД и выгрузка данных</a></span><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Обзор данных</a></span></li><li><span><a href="#Итоговый-запрос-к-БД" data-toc-modified-id="Итоговый-запрос-к-БД-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Итоговый запрос к БД</a></span></li></ul></li><li><span><a href="#Дашборд-и-презентация" data-toc-modified-id="Дашборд-и-презентация-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Дашборд и презентация</a></span></li></ul></div>

# Определение уровня лояльности клиентов

Заказчик - большая телекоммуникационная компания, оказывающая услуги на территории СНГ. 

Необходимо определить текущий уровень потребительской лояльности, или NPS, среди клиентов из России. Проведен опрос. Клиенты компании отвечали на вопрос «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым». Результаты опроса хранятся в БД SQLite в файле telecomm_csi.db.

Цели исследования: 
- составить портрет пользователей, участвующих в опросе;
- рассчитать NPS;
- составить портреть лояльного пользователя.

Задачи исследования: разработать макет дашборда и построить его в Tableau, составить отчет для клиента в формате презентации.


In [1]:
import pandas as pd
import os 
from sqlalchemy import create_engine

path_to_db = '/Users/----/----.db' # путь к файлу
path_ya = '/----.db'

if os.path.exists(path_ya): # проверка наличия файла в облаке
    engine = create_engine(f'sqlite:///{path_ya}', echo = False) #подключение к бд
else:
    try:
        engine = create_engine(f'sqlite:///{path_to_db}', echo = False)
    except:
        display('Проверьте наличие файла на локальном хранилище!')

### Обзор данных

Посмотрим на информацию из БД.

In [2]:
query = """
SELECT *
FROM location
"""
df = pd.read_sql(query, engine)
df.head()

,location_id,city,country
0,1,Архангельск,Россия
1,2,Астрахань,Россия
2,3,Балашиха,Россия
3,4,Барнаул,Россия
4,5,Белгород,Россия


In [3]:
for col in df.columns:
    print(df[col].sort_values().nunique())

62
62
1


Все клиенты из России. Количество городов и их идентификаторов совпадает и составляет 62 наименования.

In [4]:
df['city'].sort_values().unique()

array(['Архангельск', 'Астрахань', 'Балашиха', 'Барнаул', 'Белгород',
       'Брянск', 'Владивосток', 'Владимир', 'Волгоград', 'Волжский',
       'Воронеж', 'Грозный', 'Екатеринбург', 'Иваново', 'Ижевск',
       'Иркутск', 'Казань', 'Калининград', 'Калуга', 'Кемерово', 'Киров',
       'Краснодар', 'Красноярск', 'Курск', 'Липецк', 'Магнитогорск',
       'Махачкала', 'Москва', 'НабережныеЧелны', 'НижнийНовгород',
       'НижнийТагил', 'Новокузнецк', 'Новосибирск', 'Омск', 'Оренбург',
       'Пенза', 'Пермь', 'РостовнаДону', 'Рязань', 'Самара',
       'СанктПетербург', 'Саранск', 'Саратов', 'Смоленск', 'Сочи',
       'Ставрополь', 'Сургут', 'Тверь', 'Тольятти', 'Томск', 'Тула',
       'Тюмень', 'УланУдэ', 'Ульяновск', 'Уфа', 'Хабаровск', 'Чебоксары',
       'Челябинск', 'Череповец', 'Чита', 'Якутск', 'Ярославль'],
      dtype=object)

Неявных дубликатов в наименования нет.

In [5]:
query = """
SELECT *
FROM age_segment
"""
df = pd.read_sql(query, engine)
df

,age_gr_id,bucket_min,bucket_max,title
0,1,NaN,15.0,01 до 16
1,2,16.0,24.0,02 16-24
2,3,25.0,34.0,03 25-34
3,4,35.0,44.0,04 35-44
4,5,45.0,54.0,05 45-54
5,6,55.0,64.0,06 55-64
6,7,66.0,NaN,07 66 +
7,8,NaN,NaN,08 n/a


In [6]:
for col in df.columns:
    print(df[col].sort_values().nunique(), df[col].sort_values().unique())

8 [1 2 3 4 5 6 7 8]
6 [16. 25. 35. 45. 55. 66. nan]
6 [15. 24. 34. 44. 54. 64. nan]
8 ['01 до 16' '02 16-24' '03 25-34' '04 35-44' '05 45-54' '06 55-64'
 '07 66 +' '08 n/a']


Имеется 8 возрастных групп, в том числе для неуказавших возраст клиентов. 

Возраст 65 лет, вероятно по ошибке, не включен ни в одну возрастную группу. Необходимо проверить, есть ли клиенты с таким возрастом и какой идентификатор группы им проставлен.

В наименовании группы есть префикс идентификатора. Следует его удалить.

In [7]:
query = """
SELECT *
FROM traffic_segment
"""
df = pd.read_sql(query, engine)
df

,tr_gr_id,bucket_min,bucket_max,title
0,1,0.00,0.00,01 0
1,2,0.00,0.01,01 0-0.01
2,3,0.01,0.10,02 0.01-0.1
3,4,0.10,1.00,03 0.1-1
4,5,1.00,5.00,04 1-5
5,6,5.00,10.00,05 5-10
6,7,10.00,15.00,06 10-15
7,8,15.00,20.00,07 15-20
8,9,20.00,25.00,08 20-25
9,10,25.00,30.00,09 25-30


По траффику имеем 25 групп. Начальная и конечная граница групп дублируются. В описании к БД нет указания, какая граница включена в диапазон. Будем считать, что нижняя граница включена, верхняя исключена.

В наименовании группы есть префикс идентификатора. Следует его удалить.

In [8]:
query = """
SELECT *
FROM lifetime_segment
"""
df = pd.read_sql(query, engine)
df

,lt_gr_id,bucket_min,bucket_max,title
0,1,1.0,1.0,01 1
1,2,2.0,2.0,02 2
2,3,3.0,3.0,03 3
3,4,4.0,6.0,04 4-6
4,5,7.0,12.0,05 7-12
5,6,13.0,24.0,06 13-24
6,7,25.0,36.0,07 25-36
7,8,36.0,NaN,08 36+


По длительности лояльности имеем 8 групп. В седьмой группе верхняя граница дублирует значение нижей границы следующей группы. Необходимо выяснить, какой идентификатор имеют клиенты с лояльностью 36 месяцев.

В наименовании группы есть префикс идентификатора. Следует его удалить.

In [9]:
query = """
SELECT *
FROM user
"""
df = pd.read_sql(query, engine)
df.head()

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,55,5,5,8,10
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,21,5,5,8,10
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,28,6,9,6,10
3,A004TB,4190,44.0,1.0,IOS,SMARTPHONE,38,4,4,8,10
4,A004XT,1163,24.0,0.0,ANDROID,SMARTPHONE,39,2,6,8,10


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         502493 non-null  object 
 1   lt_day          502493 non-null  int64  
 2   age             501939 non-null  float64
 3   gender_segment  501192 non-null  float64
 4   os_name         502493 non-null  object 
 5   cpe_type_name   502493 non-null  object 
 6   location_id     502493 non-null  int64  
 7   age_gr_id       502493 non-null  int64  
 8   tr_gr_id        502493 non-null  int64  
 9   lt_gr_id        502493 non-null  int64  
 10  nps_score       502493 non-null  int64  
dtypes: float64(2), int64(6), object(3)
memory usage: 42.2+ MB


In [11]:
for col in df.columns:
    print(col, df[col].sort_values().nunique(), df[col].sort_values().unique())

user_id 502493 ['A001A2' 'A001WF' 'A003Q7' ... 'ZZZQ5F' 'ZZZQ8E' 'ZZZYH0']
lt_day 6950 [ -21  -13  -12 ... 8640 8828 9162]
age 79 [10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27.
 28. 29. 30. 31. 32. 33. 34. 35. 36. 37. 38. 39. 40. 41. 42. 43. 44. 45.
 46. 47. 48. 49. 50. 51. 52. 53. 54. 55. 56. 57. 58. 59. 60. 61. 62. 63.
 64. 65. 66. 67. 68. 69. 70. 71. 72. 73. 74. 75. 76. 77. 78. 79. 80. 81.
 82. 83. 84. 85. 86. 87. 89. nan]
gender_segment 2 [ 0.  1. nan]
os_name 9 ['ANDROID' 'BADA OS' 'IOS' 'OTHER' 'PROPRIETARY' 'SYMBIAN OS'
 'WINDOWS MOBILE' 'WINDOWS PHONE' 'unknown']
cpe_type_name 14 ['HANDHELD' 'MIFI ROUTER' 'MOBILE PHONE/FEATURE PHONE'
 'MOBILE TEST PLATFORM' 'MODEM' 'NETWORK DEVICE' 'PHONE'
 'PORTABLE(INCLUDE PDA)' 'ROUTER' 'SMARTPHONE' 'TABLET' 'USB MODEM'
 'WLAN ROUTER' 'unknown']
location_id 62 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52

In [12]:
df[df['lt_day'] < 0]['lt_day'].count()

13

In [13]:
df[df['lt_day'] <= 0]

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score
7666,AEAC2R,-8,NaN,NaN,ANDROID,SMARTPHONE,50,8,8,8,1
53997,CS0HF8,-2,NaN,NaN,ANDROID,SMARTPHONE,15,8,9,8,10
71577,DORPT2,-4,NaN,NaN,ANDROID,SMARTPHONE,28,8,9,8,4
100152,F5O3CG,-21,NaN,NaN,ANDROID,SMARTPHONE,60,8,12,8,7
103223,FBC993,0,NaN,NaN,ANDROID,SMARTPHONE,28,8,5,1,5
228087,LSE939,-13,NaN,NaN,ANDROID,SMARTPHONE,22,8,8,8,1
284966,OQO5GZ,0,NaN,0.0,ANDROID,SMARTPHONE,50,8,7,1,8
325212,QTT7IR,-13,NaN,NaN,IOS,SMARTPHONE,28,8,7,8,7
347784,RZUS6H,-6,NaN,NaN,ANDROID,SMARTPHONE,23,8,11,8,3
385897,TYWQW4,-12,NaN,NaN,IOS,SMARTPHONE,33,8,5,8,5


502493 записи в таблице с уникальными идентификаторами клиента, пропусков нет.

Поле "количество дней лояльности" имеет 13 записей с отрицательными значениями и 4 записи с 0 значением. Идентификатор группы лояльности для отрицательных значений присвоен максимальный, что вероятно ошибка. Абсолютные цифры не превышают 21 дня, то есть меньше месяца. Нулевые значения могут свидетельствовать о том, что клиент только что зарегистрировался.
Необходимо узнать у поставщика данных об особенностях заполнения данного поля. До выяснения целесообразно не использовать в анализе клиентов с отрицательным сроком лояльности.

В возрасте есть пропущенные значения. Необходимо заменить на "не указан". Клиенты, указавшие возраст 65 лет имеют идентификотор возрастной группы "66+". Целесообразно переименовать ее в "65+"

В гендере есть пропущенные значения. Необходимо заменить на "не указан".

Количество идентификаторов соответствует колучеству групп в идентификационных таблицах.

### Итоговый запрос к БД

In [14]:
query = """
SELECT u.user_id,
       u.lt_day,
       
       CASE 
             WHEN u.lt_day < 366 
             THEN 'да' 
             ELSE 'нет' 
       END as is_new,
       
       u.age,
       
       CASE 
             WHEN u.gender_segment = 1 
             THEN 'женщина' 
             WHEN u.gender_segment = 0 
             THEN 'мужчина'
             ELSE 'не указан' 
       END as gender_segment,
       
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       
       CASE
           WHEN SUBSTRING(a.title,4) = '66 +' 
           THEN '65+'
           WHEN SUBSTRING(a.title,4) = 'n/a'
           THEN 'не указан'
           ELSE SUBSTRING(a.title,4)
        END as age_segment,
           
       SUBSTRING(t.title,4) as traffic_segment,
       
       CASE
           WHEN SUBSTRING(lt.title,4) = '25-36'
           THEN '25-35'
           ELSE SUBSTRING(lt.title,4)
        END as lifetime_segment,
       
       u.nps_score,
       CASE 
             WHEN u.nps_score < 7 
             THEN 'критики' 
             WHEN u.nps_score > 6 and  u.nps_score < 9 
             THEN 'нейтралы'
             ELSE 'сторонники' 
       END as nps_group
       
  FROM user as u
  LEFT JOIN location as l ON u.location_id=l.location_id
  LEFT JOIN age_segment as a ON u.age_gr_id=a.age_gr_id
  LEFT JOIN traffic_segment as t ON u.tr_gr_id=t.tr_gr_id
  LEFT JOIN lifetime_segment as lt ON u.lt_gr_id=lt.lt_gr_id
  
  WHERE (u.lt_day >= 0)
"""

In [15]:
df = pd.read_sql(query, engine)
df.head()

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,нет,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,сторонники
1,A001WF,2344,нет,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,сторонники
2,A003Q7,467,нет,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,сторонники
3,A004TB,4190,нет,44.0,женщина,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,сторонники
4,A004XT,1163,нет,24.0,мужчина,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,сторонники


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502480 entries, 0 to 502479
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502480 non-null  object 
 1   lt_day            502480 non-null  int64  
 2   is_new            502480 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    502480 non-null  object 
 5   os_name           502480 non-null  object 
 6   cpe_type_name     502480 non-null  object 
 7   country           502480 non-null  object 
 8   city              502480 non-null  object 
 9   age_segment       502480 non-null  object 
 10  traffic_segment   502480 non-null  object 
 11  lifetime_segment  502480 non-null  object 
 12  nps_score         502480 non-null  int64  
 13  nps_group         502480 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


In [17]:
df[df['nps_score'] == 6].count()

user_id             16532
lt_day              16532
is_new              16532
age                 16514
gender_segment      16532
os_name             16532
cpe_type_name       16532
country             16532
city                16532
age_segment         16532
traffic_segment     16532
lifetime_segment    16532
nps_score           16532
nps_group           16532
dtype: int64

In [18]:
df.to_csv('zokham_telecomm_csi.csv', index=False)

##  Дашборд и презентация

[Дашборд на сайте Tableau Public:](https://public.tableau.com/views/zokham_talecomm_nps/Users?:language=en-US&:display_count=n&:origin=viz_share_link)